In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('../data/diabetic_data.csv')


,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,1,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,3,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,2,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),?,1,1,7,2,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,1,...,No,Steady,No,No,No,No,No,Ch,Yes,NO


In [3]:
# Drop the columns that have more than 40% null values
df.drop(columns=['weight','payer_code','medical_specialty'],inplace=True)

In [4]:
df.shape

(101766, 47)

In [5]:
# Drop duplicated row of each patient(according to 'patient_nbr')
df.drop_duplicates('patient_nbr',keep='first',inplace=True)

In [6]:
df.shape

(71518, 47)

In [7]:
# Drop rows of patient related to dead 
df_1=df[(df['discharge_disposition_id']!=11)&(df['discharge_disposition_id']!=13)&
     (df['discharge_disposition_id']!=14)&(df['discharge_disposition_id']!=19)&
     (df['discharge_disposition_id']!=20)]

In [8]:
df_1.shape

(69973, 47)

In [9]:
diabete_diag_id=['250.83','250.7','250.6','250.4','250.11','250.32','250.13','250.03','250.8','250.02','250.42','250.41','250.22',
 '250.82','250.33','250.12','250.81','250.93','250.01','250.31','250.43','250.1','250.2','250.3','250.23','250.5',
 '250.92','250.21','250.9','250.52','250.51','250.91','250.53',]

In [10]:
df_diabetes = df_1[df_1['diag_1'].map(lambda x: x in diabete_diag_id)]

In [11]:
df_diabetes.shape

(5548, 47)

In [12]:
df_cleaning = df_diabetes.copy()
# race == ?, convert to race == 'unknown'
df_cleaning['race'] = df_cleaning['race'].map(lambda x:'unknown' if x=='?' else x)
# 'gender' column, Male=1, Femail=0
df_cleaning['gender'] = df_cleaning['gender'].map({'Male':1, 'Female':0})
# convert 'age' from a range, like [50-60) to a number 55
df_cleaning['age'] = df_cleaning['age'].map({'[50-60)':55, '[60-70)':65, '[40-50)':45, '[70-80)':75, '[30-40)':35, 
                                             '[80-90)':85, '[20-30)':25, '[10-20)':15,'[0-10)':5, '[90-100)':95})
# drop one row of 'admission_type_id==4 (NewBorn)
# combine id==1,2,7 to 'Emergency', id==5,6,8 to 'Unknown', id==3 to 'Elective'
df_cleaning=df_cleaning[df_cleaning['admission_type_id']!=4]
df_cleaning['admission_type_id']=df_cleaning['admission_type_id'].map({1:'Emergency', 2:'Emergency', 3:'Elective', 
                                                                       5:'Unknown', 6:'Unknown', 8:'Unknown'})
# combine id==1 to 'dicharged home', id==2,3,4,5,15,22,23,27,28 to 'transferred to other place', 
# id==6,8,17 to 'discharged home with healthe service', id==7 to 'dama means leave hospital against advise of doctor', 
# id==18,25 to 'unknown'
df_cleaning['discharge_disposition_id'] = df_cleaning['discharge_disposition_id'].map({1:'home',2:'transferred',
3:'transferred',4:'transferred',5:'transferred',6:'home_care',7:'dama',8:'home_care',15:'transferred', 17:'home_care',
18:'unknown', 22:'transferred',23:'transferred', 25:'unknown',27:'transferred',28:'transferred'})
# column 'admission_type_id' id==7 'emergency_room', id==9,17,20 'unknown', id==1,2,3,8 'referral', id==4,5,6 'transfered'
df_cleaning['admission_source_id'] = df_cleaning['admission_source_id'].map(
{7:'emergency_room', 1:'referral', 17:'unknown', 6:'transfered', 4:'transfered', 2:'referral', 5:'transfered', 
 20:'unknown', 9:'unknown', 3:'referral', 8:'referral'})
# transform time_in_hospital to log(time_in_hospital)
df_cleaning['time_in_hospital']=df_cleaning['time_in_hospital'].map(lambda x: np.log(1+x))
# no change for column 'num_lab_procedures'
# no change for column 'num_precedures'
# transform column 'num_medications' with np.log()
df_cleaning['num_medications']=df_cleaning['num_medications'].map(lambda x: np.log(1+x))
# 'number_outpatient', 'number_emergency' and 'number_inpatient' are all dominated by one value==0
# drop off column 'diag_1' 'diag_2' and diag_3
df_cleaning.drop(columns=['diag_1','diag_2','diag_3'],inplace=True)
# 'number_diagnoses' not normally distributed, but log won't work, so didn't make any changes
# column 'chlorpropamide', 'acetohexamide', 'miglitol', 'troglitazone','examide','citoglipton','glipizide-metformin',
# column 'glimepiride-pioglitazone','metformin-rosiglitazone','metformin-pioglitazone', all data give one value
df_cleaning.drop(columns=['chlorpropamide', 'acetohexamide', 'miglitol', 'troglitazone','examide','citoglipton',
                          'glipizide-metformin','glimepiride-pioglitazone','metformin-rosiglitazone',
                          'metformin-pioglitazone'], inplace=True)
# column 'max_glu_serum' glucose serum test result, None==no measure==0, Norm==<100==1, >200==2, >300==3
df_cleaning['max_glu_serum']=df_cleaning['max_glu_serum'].map({'None':0, '>300':3,'Norm':1,'>200':2})
# column 'ALCresult' related to diabetes, None==0, Norm==<5.7==1, >7==2, >8==3
df_cleaning['A1Cresult']=df_cleaning['A1Cresult'].map({'None':0, 'Norm':1, '>7':2, '>8':3})

In [ ]:
categorical_columns=['race','gender','admission_type_id','discharge_disposition_id','admission_source_id','change',
                     'diabetesMed','diabetesMed_change']
numerical_columns=['age','time_in_hospital','num_lab_procedures','num_procedures','num_medications','number_outpatient', 
                   'number_emergency','number_inpatient','number_diagnoses','max_glu_serum','A1Cresult',
                   'number_features_medications']

In [60]:
features=['metformin', 'repaglinide', 'nateglinide','glimepiride','glipizide','glyburide',
'pioglitazone','rosiglitazone','acarbose','insulin', 'glyburide-metformin','tolbutamide','tolazamide']


In [69]:
df_cleaning['number_features_medications']=0

In [71]:
for i in range(len(df_cleaning)):
    count = 0
    for column_name in features:
        if df_cleaning[column_name][i] != 'No':
            count+=1
    df_cleaning['number_features_medications'][i] = count

/Users/susanchen/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [80]:
df_cleaning['diabetesMed_change']=df_cleaning['diabetesMed']+df_cleaning['change']

In [88]:
df_cleaning['label'] = df_cleaning['readmitted'].map({'<30':1, '>30':0, 'NO':0})

In [91]:
df_cleaning.shape

(5547, 37)

In [92]:
df_cleaning.to_csv('../data/UCI_train_test.csv')